In [ ]:
# Following https://github.com/huggingface/notebooks/blob/master/examples/translation.ipynb

In [1]:
!pip install wandb -qqq

     |████████████████████████████████| 1.7 MB 12.4 MB/s 
     |████████████████████████████████| 139 kB 49.3 MB/s 
     |████████████████████████████████| 97 kB 6.7 MB/s 
     |████████████████████████████████| 180 kB 43.5 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 


In [4]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [5]:
!nvidia-smi

Sun Oct  3 14:10:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [44]:
import datetime
model_checkpoint = 't5-small'
fp16 = True
todaydate = datetime.datetime.now().strftime("%Y-%m-%d")
print(todaydate)
gdir = f'drive/My Drive/Colab Notebooks/{todaydate}/'
model_name=f'sparql-translator-{todaydate}-{model_checkpoint}' + '-fp16' if fp16 else ''
model_path='./models/'+model_name
ds_path= 'lc-quad-wikidata-2021-10-03'
ds_name = 'yazdipour/text-to-sparql-t5-lc-quad-v2'

2021-10-03


In [18]:
!unzip {ds_path}.zip

Archive:  lc-quad-wikidata-2021-10-03.zip
   creating: lc-quad-wikidata-2021-10-03/
 extracting: lc-quad-wikidata-2021-10-03/dataset_dict.json  
   creating: lc-quad-wikidata-2021-10-03/test/
  inflating: lc-quad-wikidata-2021-10-03/test/state.json  
  inflating: lc-quad-wikidata-2021-10-03/test/dataset_info.json  
  inflating: lc-quad-wikidata-2021-10-03/test/dataset.arrow  
   creating: lc-quad-wikidata-2021-10-03/train/
  inflating: lc-quad-wikidata-2021-10-03/train/state.json  
  inflating: lc-quad-wikidata-2021-10-03/train/dataset_info.json  
  inflating: lc-quad-wikidata-2021-10-03/train/dataset.arrow  


In [45]:
# Flexible integration for any Python script
import wandb

# 1. Start a W&B run
wandb.init(project='text-to-sparql', entity='shahriar', name= f'{todaydate}-{model_checkpoint}')

# 2. Save model inputs and hyperparameters
config = wandb.config
config.learning_rate = 0.01

eval/bleu,▁
eval/gen_len,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▄▆██
train/global_step,▁▄▆██
train/learning_rate,█▅▁
train/loss,█▂▁
train/total_flos,▁


In [46]:
model_name

'sparql-translator-2021-10-03-t5-small-fp16'

In [47]:
!mkdir models

mkdir: cannot create directory ‘models’: File exists


In [11]:
!pip install datasets transformers sacrebleu==1.5.1 -qqq

     |████████████████████████████████| 270 kB 13.0 MB/s 
     |████████████████████████████████| 2.9 MB 33.0 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 1.3 MB 38.8 MB/s 
     |████████████████████████████████| 125 kB 52.7 MB/s 
     |████████████████████████████████| 243 kB 46.6 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 636 kB 30.9 MB/s 
     |████████████████████████████████| 3.3 MB 37.1 MB/s 
     |████████████████████████████████| 895 kB 40.5 MB/s 
     |████████████████████████████████| 142 kB 48.4 MB/s 
     |████████████████████████████████| 294 kB 44.5 MB/s 


In [12]:
from datasets import load_dataset, load_metric, Dataset, load_from_disk

In [19]:
raw_datasets = load_from_disk(ds_path)

In [ ]:
!pip install tqdm==4.49.0

     |████████████████████████████████| 69 kB 4.8 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.3
    Uninstalling tqdm-4.62.3:
      Successfully uninstalled tqdm-4.62.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 1.12.1 requires tqdm>=4.62.1, but you have tqdm 4.49.0 which is incompatible.


In [20]:
from tqdm import tqdm

In [21]:
metric = load_metric("sacrebleu")

Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

In [ ]:
# Preprocessing

In [48]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Could not locate the tokenizer configuration file, will try to use the model config instead.
https://huggingface.co/t5-small/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpbim92rnx


Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

storing https://huggingface.co/t5-small/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
creating metadata file for /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "laye

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

storing https://huggingface.co/t5-small/resolve/main/spiece.model in cache at /root/.cache/huggingface/transformers/65fc04e21f45f61430aea0c4fedffac16a4d20d78b8e6601d8d996ebefefecd2.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d
creating metadata file for /root/.cache/huggingface/transformers/65fc04e21f45f61430aea0c4fedffac16a4d20d78b8e6601d8d996ebefefecd2.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d
https://huggingface.co/t5-small/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpu9_y9cnu


Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

storing https://huggingface.co/t5-small/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/06779097c78e12f47ef67ecb728810c2ae757ee0a9efe9390c6419783d99382d.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529
creating metadata file for /root/.cache/huggingface/transformers/06779097c78e12f47ef67ecb728810c2ae757ee0a9efe9390c6419783d99382d.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529
loading file https://huggingface.co/t5-small/resolve/main/spiece.model from cache at /root/.cache/huggingface/transformers/65fc04e21f45f61430aea0c4fedffac16a4d20d78b8e6601d8d996ebefefecd2.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d
loading file https://huggingface.co/t5-small/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/06779097c78e12f47ef67ecb728810c2ae757ee0a9efe9390c6419783d99382d.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529
loading file https://huggingface.co/t5-small/reso

In [26]:
print(raw_datasets['test']['translation'][0]['sparql'],'\n',
      raw_datasets['test']['translation'][0]['en'],'\n',
      tokenizer(raw_datasets['test']['translation'][0]['en']))

select ?value where [ wd:alec_guinness p:nominated_for ?s . ?s ps:nominated_for wd:best_actor . ?s pq:point_in_time ?value] 
 When Alec Guinness was nominated as Academy Award for Best Actor? 
 {'input_ids': [366, 15345, 75, 2846, 77, 655, 47, 150, 1109, 920, 38, 4702, 3677, 21, 1648, 1983, 127, 58, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [32]:
max_input_length = 0 
max_target_length = 0
for d in tqdm(raw_datasets['train']['translation']):
    len_en = len(d['en'])
    len_qry = len(d['sparql'])
    if len_en > max_input_length: max_input_length=len_en
    if len_qry > max_target_length: max_target_length=len_qry
print('\n',max_input_length, max_target_length)

100%|██████████| 15123/15123 [00:00<00:00, 776192.60it/s]


 243 273


In [33]:
source_lang = "en"
target_lang = "sparql"
prefix = "translate English to Sparql: "

def preprocess_function(examples):
    inputs = []
    targets= []
    for ex in examples["translation"]:
      inputs.append(prefix + ex[source_lang])
      targets.append(ex[target_lang])
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
      labels = tokenizer(targets, max_length=max_target_length, truncation=True)
      
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [34]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [35]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 15123
    })
    test: Dataset({
        features: ['__index_level_0__', 'attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 3781
    })
})

In [ ]:
# Fine-tuning the model

In [49]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "pre

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

storing https://huggingface.co/t5-small/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/fee5a3a0ae379232608b6eed45d2d7a0d2966b9683728838412caccc41b4b0ed.ddacdc89ec88482db20c676f0861a336f3d0409f94748c209847b49529d73885
creating metadata file for /root/.cache/huggingface/transformers/fee5a3a0ae379232608b6eed45d2d7a0d2966b9683728838412caccc41b4b0ed.ddacdc89ec88482db20c676f0861a336f3d0409f94748c209847b49529d73885
loading weights file https://huggingface.co/t5-small/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/fee5a3a0ae379232608b6eed45d2d7a0d2966b9683728838412caccc41b4b0ed.ddacdc89ec88482db20c676f0861a336f3d0409f94748c209847b49529d73885
All model checkpoint weights were used when initializing T5ForConditionalGeneration.

All the weights of T5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use 

In [50]:
batch_size = 8
args = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [51]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [39]:
# The last thing to define for our Seq2SeqTrainer is how to compute 
# the metrics from the predictions. We need to define 
# a function for this, which will just use the metric we loaded earlier, 
# and we have to do a bit of pre-processing to decode the predictions into texts:

import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.replace('?',' ?').replace('.', ' .').strip() for pred in preds]
    labels = [[label.replace('?',' ?').replace('.', ' .').strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [52]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [53]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation, __index_level_0__.
***** Running training *****
  Num examples = 15123
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1891
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.143500,0.702415,3.042500,19.000000


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1349: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
Saving model checkpoint to sparql-translator-2021-10-03-t5-small-fp16/checkpoint-500
Configuration saved in sparql-translator-2021-10-03-t5-small-fp16/checkpoint-500/config.json
Model weights saved in sparql-translator-2021-10-03-t5-small-fp16/checkpoint-500/pytorch_model.bin
tokenizer config file saved in sparql-translator-2021-10-03-t5-small-fp16/checkpoint-500/tokenizer_config.json
Special tokens file saved in sparql-translator-2021-10-03-t5-small-fp16/checkpoint-500/special_tokens_map.json
Saving model checkpoint to sparql-translator-2021-10-03-t5-small-fp16/checkpoint-1000
Configuration s

TrainOutput(global_step=1891, training_loss=1.6020427120860325, metrics={'train_runtime': 1139.108, 'train_samples_per_second': 13.276, 'train_steps_per_second': 1.66, 'total_flos': 579664706273280.0, 'train_loss': 1.6020427120860325, 'epoch': 1.0})

In [54]:
trainer.save_model(model_path)

Saving model checkpoint to ./models/sparql-translator-2021-10-03-t5-small-fp16
Configuration saved in ./models/sparql-translator-2021-10-03-t5-small-fp16/config.json
Model weights saved in ./models/sparql-translator-2021-10-03-t5-small-fp16/pytorch_model.bin
tokenizer config file saved in ./models/sparql-translator-2021-10-03-t5-small-fp16/tokenizer_config.json
Special tokens file saved in ./models/sparql-translator-2021-10-03-t5-small-fp16/special_tokens_map.json


In [55]:
!ls -l --block-size=M {model_path}

total 233M
-rw-r--r-- 1 root root   1M Oct  3 15:56 config.json
-rw-r--r-- 1 root root 231M Oct  3 15:56 pytorch_model.bin
-rw-r--r-- 1 root root   1M Oct  3 15:56 special_tokens_map.json
-rw-r--r-- 1 root root   1M Oct  3 15:56 tokenizer_config.json
-rw-r--r-- 1 root root   2M Oct  3 15:56 tokenizer.json
-rw-r--r-- 1 root root   1M Oct  3 15:56 training_args.bin


In [56]:
!mkdir drive/MyDrive/models/{model_name}

In [57]:
!cp {model_path}/* drive/MyDrive/models/{model_name}/

In [42]:
data_files = {"train": "train.csv", "test": "test.csv"}
dataset = load_dataset("yazdipour/text-to-sparql-t5-lc-quad-v2", data_files=data_files)

Using custom data configuration text-to-sparql-t5-lc-quad-v2-25e91425efd0ac0c


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/3.27M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/text-to-sparql-t5-lc-quad-v2-25e91425efd0ac0c/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [43]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'translation', '__index_level_0__'],
        num_rows: 15123
    })
    test: Dataset({
        features: ['Unnamed: 0', 'translation', '__index_level_0__'],
        num_rows: 3781
    })
})